
# Lab 9 Validation Notebook

You cannot write any code in this notebook – to actually work on the lab, open `lab.py` and `lab.ipynb`.

The only purpose of this notebook is to give you a blank copy of `lab.ipynb`,
so that you can go to **Kernel > Restart & Run All** and ensure that all public `grader.check` cells pass using just the code in your `lab.py`.

**Before submitting Lab 9, make sure that the call to `grader.check_all()` at the bottom of this notebook shows that all test cases passed!** 
If it doesn't, there's likely a function in `lab.ipynb` that is not implemented correctly in `lab.py`, or it could be that a function in `lab.py` depends on an object (e.g. a DataFrame) that is not an argument to that function.
    

In [75]:
# Initialize Otter
import otter
grader = otter.Notebook("lab.ipynb")

In [76]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [77]:
from lab import *

In [78]:
import pandas as pd
import numpy as np
import os
import plotly.express as px

from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import FunctionTransformer
from sklearn.preprocessing import OneHotEncoder
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer

from pipeline_testing_util import get_transformers

from plotly.subplots import make_subplots
import plotly.graph_objects as go

import warnings
warnings.simplefilter('ignore')

In [79]:
fp = os.path.join('data', 'toy.csv')
data = pd.read_csv(fp)
data.head()

,group,c1,c2,y
0,B,4.058118,5.329582,12.649035
1,A,4.194945,3.839476,17.309083
2,A,2.246411,10.694666,15.695646
3,B,2.510912,6.414960,11.535752
4,B,3.194722,6.116839,14.954389


In [80]:
# don't change this cell, but do run it -- it is needed for the tests to work
q1_fp = os.path.join('data', 'toy.csv')
q1_data = pd.read_csv(q1_fp)
q1_pl, q1_preds = simple_pipeline(q1_data)

In [81]:
grader.check("q1")

q1 results: All test cases passed!

In [82]:
# don't change this cell, but do run it -- it is needed for the tests to work
q2_fp = os.path.join('data', 'toy.csv')
q2_data = pd.read_csv(q2_fp)
q2_pl, q2_preds = multi_type_pipeline(q2_data)

In [83]:
grader.check("q2")

q2 results: All test cases passed!

In [84]:
# The scatter plot referenced at the start of Question 3
# This is not needed to answer the question, but motivates why we are standardizing
px.scatter(data, x='c1', y='y', color='group')

In [85]:
# don't change this cell, but do run it -- it is needed for the tests to work
# test fit 
q3_test_fit_cols = {'g': ['A', 'A', 'B', 'B'], 'c1': [1, 2, 2, 2], 'c2': [3, 1, 2, 0]}
q3_test_fit_X = pd.DataFrame(q3_test_fit_cols)
q3_test_fit_std = StdScalerByGroup().fit(q3_test_fit_X)

# test transform
q3_test_transform_cols = {'g': ['A', 'A', 'B', 'B'], 'c1': [1, 2, 3, 4], 'c2': [1, 2, 3, 4]}
q3_test_transform_X = pd.DataFrame(q3_test_transform_cols)
q3_test_transform_std = StdScalerByGroup().fit(q3_test_transform_X)
q3_test_transform_out = q3_test_transform_std.transform(q3_test_transform_X)

In [86]:
# don't change this cell, but do run it -- it is needed for the tests to work
q3_fit_data = pd.read_csv('data/toy.csv')

N = 2*10**6
a = np.random.choice(['A', 'B'], size=(N,1)).astype('object')
b = np.random.multivariate_normal([1, 2], [[1, 0],[0, 100]], size=N)
arr = np.hstack([a, b])
q3_transform_data = pd.DataFrame(arr)
q3_transform_data[1] = q3_transform_data[1].astype(float)
q3_transform_data[2] = q3_transform_data[2].astype(float)

In [87]:
grader.check("q3")

q3 results: All test cases passed!

In [88]:
grader.check("q4")

q4 results: All test cases passed!

In [89]:
# Use `galton` to test your work.
galton = pd.read_csv('data/galton.csv')
galton.head()

,father,mother,children,childNum,gender,childHeight
0,78.5,67.0,4,1,1,73.2
1,78.5,67.0,4,2,0,69.2
2,78.5,67.0,4,3,0,69.0
3,78.5,67.0,4,4,0,69.0
4,75.5,66.5,4,1,1,73.5


In [90]:
# don't change this cell, but do run it -- it is needed for the tests
galton_test = pd.read_csv('data/galton.csv')
out_tree_test = tree_reg_perf(galton_test)
out_knn_test = knn_reg_perf(galton_test)

In [91]:
grader.check("q5")

q5 results: All test cases passed!

In [114]:
np.random.seed(9) # For reproducibility

tree = tree_reg_perf(galton)
knn = knn_reg_perf(galton)
hyp = np.arange(1, 21)

fig = make_subplots(rows=1, cols=2, subplot_titles=('Error vs. Tree Depth<br>for Decision Tree Regressor',
                                                    'Error vs. k (# Neighbors)<br>for k-NN Regressor'))

fig.add_trace(
    go.Scatter(x=hyp, y=tree.iloc[:, 0], line=dict(color='blue'), name='Training Error'),
    row=1, col=1).add_trace(go.Scatter(x=hyp, y=tree.iloc[:, 1], line=dict(color='red'), name='Testing Error'), 
                            row=1, col=1)

fig.add_trace(
    go.Scatter(x=hyp, y=knn.iloc[:, 0], line=dict(color='blue'), name='Training Error', showlegend=False),
    row=1, col=2).add_trace(go.Scatter(x=hyp, y=knn.iloc[:, 1], line=dict(color='red'),  name='Testing Error',
                                       showlegend=False), row=1, col=2)

fig.update_layout(height=450, width=975)
fig.update_xaxes(title_text='Tree Depth', row=1, col=1, tickvals=np.arange(1,21,2))
fig.update_xaxes(title_text='k (# Neighbors)', row=1, col=2, tickvals=np.arange(1,21,2))

fig.show()

In [65]:
# Experiment using `titanic` below – remember, this is only your training data
titanic = pd.read_csv('data/titanic.csv')
titanic.head()

,Survived,Pclass,Name,Sex,Age,Siblings/Spouses Aboard,Parents/Children Aboard,Fare
0,0,2,Mr. William John Berriman,male,23.0,0,0,13.0000
1,1,3,Mrs. (Beila) Moor,female,27.0,0,1,12.4750
2,0,3,Mr. Nestor Cyriel Vande Walle,male,28.0,0,0,9.5000
3,0,3,Mr. Khalil Saad,male,25.0,0,0,7.2250
4,0,3,Miss. Gerda Ulrika Dahlberg,female,22.0,0,0,10.5167


In [66]:
from sklearn.model_selection import train_test_split
X = titanic.drop(columns = 'Survived')
y = titanic['Survived']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.35)
pre = titanic_model(titanic)
(pre.predict(X_test)==y_test).mean()

0.8634538152610441

In [67]:
# don't change this cell, but do run it -- it is needed for the tests
q6_data_test = pd.read_csv('data/titanic.csv')
pl_test = titanic_model(q6_data_test)

In [68]:
grader.check("q6")

q6 results: All test cases passed!

In [69]:
grader.check_all()

q1 results: All test cases passed!

q2 results: All test cases passed!

q3 results: All test cases passed!

q4 results: All test cases passed!

q5 results: All test cases passed!

q6 results: All test cases passed!


If you were able to go to **Kernel > Restart & Run All** and see all test cases pass above, and you've thoroughly tested your code yourself, you're ready to submit `lab.py` to Gradescope!
    